In [ ]:
#install library from other
!pip install mediapipe
!pip install neurokit2

In [3]:
# import library
import mediapipe as mp
import cv2
import numpy as np
from scipy.signal import butter, sosfiltfilt, find_peaks, welch
from statistics import multimode,mean
from neurokit2.signal import signal_smooth
from scipy.stats import entropy
from scipy.interpolate import interp1d
from scipy.integrate import trapz

In [ ]:
# Download and un-zip ex videos
! gdown 1rsaa8Q0vQmkmzlLLhMkFo-qpLWNM-GC5
!unzip /content/Example_video.zip
# Download and un-zip model
! gdown 1imOtgGaV88ij8LcC3eJMOPQGtjFLgzTf
!unzip /content/NN_model.zip

In [4]:
#import tools from mediapipe
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

#select the (face) model type from mediapipe
mp_face_mesh = mp.solutions.face_mesh
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

In [5]:
# define forehead area by selecting the interest points on subject's face
def forehead_define():
    forehead_1 = np.array((landmark_points[103], landmark_points[67], landmark_points[104]))
    forehead_2 = np.array((landmark_points[67], landmark_points[104], landmark_points[69]))
    forehead_3 = np.array((landmark_points[108], landmark_points[69], landmark_points[67]))
    forehead_4 = np.array((landmark_points[109], landmark_points[108], landmark_points[67]))
    forehead_5 = np.array((landmark_points[109], landmark_points[151], landmark_points[108]))
    forehead_6 = np.array((landmark_points[10], landmark_points[109], landmark_points[151]))
    forehead_7 = np.array((landmark_points[10], landmark_points[151], landmark_points[338]))
    forehead_8 = np.array((landmark_points[151], landmark_points[337], landmark_points[338]))
    forehead_9 = np.array((landmark_points[338], landmark_points[337], landmark_points[297]))
    forehead_10 = np.array((landmark_points[337], landmark_points[299], landmark_points[297]))
    forehead_11 = np.array((landmark_points[297], landmark_points[299], landmark_points[333]))
    forehead_12 = np.array((landmark_points[297], landmark_points[333], landmark_points[332]))
    forehead_13 = np.array((landmark_points[104], landmark_points[69], landmark_points[105]))
    forehead_14 = np.array((landmark_points[105], landmark_points[66], landmark_points[69]))
    forehead_15 = np.array((landmark_points[69], landmark_points[66], landmark_points[107]))
    forehead_16 = np.array((landmark_points[108], landmark_points[69], landmark_points[107]))
    forehead_17 = np.array((landmark_points[108], landmark_points[107], landmark_points[9]))
    forehead_18 = np.array((landmark_points[151], landmark_points[108], landmark_points[9]))
    forehead_19 = np.array((landmark_points[151], landmark_points[9], landmark_points[337]))
    forehead_20 = np.array((landmark_points[9], landmark_points[336], landmark_points[337]))
    forehead_21 = np.array((landmark_points[337], landmark_points[336], landmark_points[299]))
    forehead_22 = np.array((landmark_points[336], landmark_points[296], landmark_points[299]))
    forehead_23 = np.array((landmark_points[299], landmark_points[296], landmark_points[334]))
    forehead_24 = np.array((landmark_points[299], landmark_points[334], landmark_points[333]))
    return forehead_1,forehead_2,forehead_3,forehead_4,forehead_5,forehead_6,forehead_7,forehead_8,forehead_9,forehead_10,forehead_11,forehead_12,forehead_13,forehead_14,forehead_15,forehead_16,forehead_17,forehead_18,forehead_19,forehead_20,forehead_21,forehead_22,forehead_23,forehead_24

# define mask
def mask_define(h,w):
    mask = np.zeros((h, w), dtype=np.uint8)
    return mask

# define BP filter
def bandpass_filter(data, lowcut, highcut,fs):
    nyq = 0.5 * fs
    low = float(lowcut) / float(nyq)
    high = float(highcut) / float(nyq)
    order = 3.0
    sos = butter(order, [low, high], btype='band',output='sos')
    #Change filt or filtfilt here
#     bandpass = sosfilt(sos, data)
    bandpass = sosfiltfilt(sos, data)
    return bandpass

# define modified POS
def POS(red, green, blue, frame, low, high,fs):
    win_size = 30
    H = np.zeros(frame)
    idx = 0
    while True:
        R_interval_norm = red[idx:idx+win_size]/red[idx:idx+win_size].mean()
        G_interval_norm = green[idx:idx+win_size]/green[idx:idx+win_size].mean()
        B_interval_norm = blue[idx:idx+win_size]/blue[idx:idx+win_size].mean()
        color_arr = np.array((R_interval_norm, G_interval_norm, B_interval_norm))
        S_1 = (-0.168*R_interval_norm) - (0.331*G_interval_norm) + (0.499*B_interval_norm)
        S_2 = (0.499*R_interval_norm) -  (0.418*G_interval_norm) - (0.081*B_interval_norm)
        alpha = S_1.std()/S_2.std()
        h = S_1 + (alpha * S_2)
        H[idx:idx+win_size] = H[idx:idx+win_size] + (h - h.mean())
        if idx+win_size > frame:
            break
        idx = idx + 1
    return bandpass_filter(H,low,high,fs)

# Extract the number of sample points in each area from the whole data
def extract_data(red_arr,green_arr,blue_arr,idx,sample_point_num,area_num):
    tmp_signal_red,tmp_signal_green, tmp_signal_blue= [],[],[]
    red_color_arr, green_color_arr, blue_color_arr = [],[],[]
    for i in range(area_num):
        tmp_signal_red.append(red_arr[i][idx:idx+sample_point_num])
        tmp_signal_green.append(green_arr[i][idx:idx+sample_point_num])
        tmp_signal_blue.append(blue_arr[i][idx:idx+sample_point_num])

        red_now = np.array(tmp_signal_red.copy())
        green_now = np.array(tmp_signal_green.copy())
        blue_now = np.array(tmp_signal_blue.copy())

        red_color_arr.append(red_now)
        green_color_arr.append(green_now)
        blue_color_arr.append(blue_now)

        tmp_signal_red.clear()
        tmp_signal_green.clear()
        tmp_signal_blue.clear()
    return red_color_arr,green_color_arr,blue_color_arr

# reshape data
def reshape_data(red,green,blue,area_num,sample_point_num):
    red_color_arr_np = np.array(red.copy())
    green_color_arr_np = np.array(green.copy())
    blue_color_arr_np = np.array(blue.copy())
    red_color_arr_np = np.reshape( red_color_arr_np,(area_num,sample_point_num))
    green_color_arr_np = np.reshape( green_color_arr_np,(area_num,sample_point_num))
    blue_color_arr_np = np.reshape( blue_color_arr_np,(area_num,sample_point_num))
    return red_color_arr_np,green_color_arr_np,blue_color_arr_np

#Find hair area
def find_hair(red_arr,green_arr,blue_arr):
    red_no_hair,green_no_hair , blue_no_hair = [],[],[]
    max_green = np.max(green_arr.mean(axis=1))

    #Find the average value of green color in each area
    tmp_green = green_arr.mean(axis=1)

    #Find which area is the hair
    hair_area_green = np.where(tmp_green/max_green<0.7)

    # Find which index is the hair then remove it
    for i in range(len(green_arr)):
        if (i == hair_area_green[0]).any():
            continue
        else:
            red_no_hair.append(red_arr[i])
            green_no_hair.append(green_arr[i])
            blue_no_hair.append(blue_arr[i])
    red_no_hair_arr = np.array(red_no_hair)
    green_no_hair_arr = np.array(green_no_hair)
    blue_no_hair_arr = np.array(blue_no_hair)
    return red_no_hair_arr,green_no_hair_arr,blue_no_hair_arr

#Find rppg
def find_rppg(red,green,blue,low,high,fs,area_num,sample_point_num):
    signal = []
    for i in range(area_num):
        S_pos = POS(red[i], green[i], blue[i],sample_point_num,low,high,fs)
        signal.append(S_pos)
        signal_arr = np.array(signal.copy())
    # Find average value of the rPPG
    S_pos_avg_normal = signal_arr.mean(axis=0)
    return S_pos_avg_normal

#Find hr
def find_hr(S_pos,fft_point,df):
    s_f = 20*np.log10(np.abs(np.fft.fft(S_pos,n=fft_point) + pow(10,-10)))
    hr = np.argmax(s_f[:fft_point//2]) * df * 60
    return hr

#Find group hr (Majority vote)
def find_group(hr):
    num_div = 5
    while True:
        hr_tmp_new = (np.array(hr)//num_div) * num_div
        mode_hr = multimode(hr_tmp_new)
        mode_hr_arr = np.array(mode_hr)
        if len(mode_hr_arr) == 1:
            mode_val = mode_hr_arr
            break
        elif len(mode_hr_arr) > 2:
            num_div += 5
        else:
            if np.abs(np.diff(mode_hr_arr)) == num_div:
                mode_val = mode_hr_arr.mean()
                break
            else:
                num_div += 5
    return mode_val,num_div

# Peak detection from neurokit 2
def ppg_findpeaks_elgendi(
    signal,
    sampling_rate,
    peakwindow=0.111, #0.111
    beatwindow=0.667, #0.667
    beatoffset=0.02, #0.02
    mindelay=0.25, #0.3
    show=False,
):
    # Ignore the samples with negative amplitudes and square the samples with
    # values larger than zero.
    signal_abs = signal.copy()
    signal_abs[signal_abs < 0] = 0
    sqrd = signal_abs ** 2

    # Compute the thresholds for peak detection. Call with show=True in order
    # to visualize thresholds.
    ma_peak_kernel = int(np.rint(peakwindow * sampling_rate))
    ma_peak = signal_smooth(sqrd, kernel="boxcar", size=ma_peak_kernel)

    ma_beat_kernel = int(np.rint(beatwindow * sampling_rate))
    ma_beat = signal_smooth(sqrd, kernel="boxcar", size=ma_beat_kernel)

    thr1 = ma_beat + beatoffset * np.mean(sqrd)  # threshold 1

    # Identify start and end of PPG waves.
    waves = ma_peak > thr1
    beg_waves = np.where(np.logical_and(np.logical_not(waves[0:-1]), waves[1:]))[0]
    end_waves = np.where(np.logical_and(waves[0:-1], np.logical_not(waves[1:])))[0]
    # Throw out wave-ends that precede first wave-start.
    end_waves = end_waves[end_waves > beg_waves[0]]

    # Identify systolic peaks within waves (ignore waves that are too short).
    num_waves = min(beg_waves.size, end_waves.size)
    min_len = int(
        np.rint(peakwindow * sampling_rate)
    )  # this is threshold 2 in the paper
    min_delay = int(np.rint(mindelay * sampling_rate))
    peaks = []

    for i in range(num_waves):

        beg = beg_waves[i]
        end = end_waves[i]
        len_wave = end - beg

        if len_wave < min_len:
            continue

        # Find local maxima and their prominence within wave span.
        data = signal[beg:end]
        locmax, props = find_peaks(data, prominence=(None, None))

        if locmax.size > 0:
            # Identify most prominent local maximum.
            peak = beg + locmax[np.argmax(props["prominences"])]
#             print(peaks,i)
            # Enforce minimum delay between peaks.
            if i == 0 or len(peaks) == 0:
                peaks.append(peak)
            else:
                if peak - peaks[-1] > min_delay:
                    peaks.append(peak)

    peaks = np.asarray(peaks).astype(int)
    return peaks

def prv(peak,fs):
    #convert peak idx to milisecond
    rri = (np.diff(peak)/fs) *1000
    dif_rri = np.diff(rri)
    mean_pp = rri.mean()
    rmssd = np.sqrt(pow(dif_rri,2).mean())
    sdnn = np.std(rri,ddof=1)
    sdsd = np.std(dif_rri,ddof=1)
    return mean_pp,rmssd,sdnn,sdsd

def prv_nonlinear(peak,fs):
    #sd1,sd2
    rri = (np.diff(peak)/fs) *1000
    rri_n = rri[:-1]
    rri_p = rri[1:]
    x1 = (rri_n - rri_p) / np.sqrt(2)
    x2 = (rri_n + rri_p) / np.sqrt(2)
    sd1 = np.std(x1)
    sd2 = np.std(x2)
    #ShanEn
    freq = np.unique(rri)
    Shan = entropy(freq,base=2)
    return sd1,sd2,Shan

def interpolate(peak,fs):
    rri = (np.diff(peak)/fs) *1000
    steps = 1/fs
    x = np.cumsum(rri) / 1000.0
    f = interp1d(x, rri, fill_value="extrapolate", kind='cubic')
    xx = np.arange(1, np.max(x), steps)
    rri_interpolated = f(xx)
    return rri_interpolated

def frequency_domain(rri, fs=30):
    # Estimate the spectral density using Welch's method
    fxx, pxx = welch(x=rri, fs=fs, nperseg=len(rri), nfft=3600)

    '''
    Segement found frequencies in the bands
     - Very Low Frequency (VLF): 0-0.04Hz
     - Low Frequency (LF): 0.04-0.15Hz
     - High Frequency (HF): 0.15-0.4Hz
    '''
#     cond_vlf = (fxx >= 0) & (fxx < 0.04)
    cond_lf = (fxx >= 0.04) & (fxx < 0.15)
    cond_hf = (fxx >= 0.15) & (fxx < 0.4)

    # calculate power in each band by integrating the spectral density
#     vlf = trapz(pxx[cond_vlf], fxx[cond_vlf])
    lf = trapz(pxx[cond_lf], fxx[cond_lf])
    hf = trapz(pxx[cond_hf], fxx[cond_hf])

    # sum these up to get total power
#     total_power = vlf + lf + hf
    return hf,lf, lf/hf

def add_data_s(data,index):
    time = 10
    tmp = data[index:time+index]
    return mean(tmp)

def check_predict(y):
    if y < 0.5:
        predict_re = "Alert"
    else:
        predict_re = "Drowsy"
    return predict_re

**Model**

In [ ]:
# Load model data
from tensorflow.keras.models import load_model
from joblib import load

NN_model = load_model('/content/model_11fea_90%.h5')
# import the mean and std of training data for scaling the data
sc = load('/content/sc_90%.bin')
NN_model.summary()

**Main**

In [ ]:
interval_time = 20 #length of each interval used to extract rPPG from the video
subinterval_time = 6 #length of each sub interval used to extract hr from 20s rPPG to do majority vote
frame_time = 30*interval_time #number of video frame used to extract rPPG (FPS x time)
fft_point = 1800
df = 30/fft_point

#set up your video path
data_path = "/content/alert.avi"
# data_path = "/content/drowsy.avi"

# open video
cap = cv2.VideoCapture(data_path)
fps = cap.get(cv2.CAP_PROP_FPS)

#list use for containing variable
red_f, green_f, blue_f= [],[],[]
red_var, green_var, blue_var = [],[],[]
idx, data_idx = 0,0
count_hr,count_mpp = 0,0

# HR
tmp_hr, hr_lst, tmp_mpp = [],[],[]
# rPPG before and after majority mode
S_pos_tmp, S_pos_before= [],[]
#prv
meanPP,RMSSD,sdNN,SDSD = [],[],[],[]
sd1_lst,sd2_lst,sd1_sd2,shanen = [],[],[],[]
hf,lf, lhf = [],[],[]
# counting predict result
count_a,count_d = 0,0

# MediaPipe face mesh model
with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh: #setup face detection parameter

    while cap.isOpened():
        success, image = cap.read()
        if success == False:
            break
        height, width, _ = image.shape
        if success:
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = face_mesh.process(image)

        # Draw the face mesh annotations on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            if results.multi_face_landmarks:
                for face_landmarks in results.multi_face_landmarks:

                # convert normalization coordinate into normal coordinate
                    landmark_points = []
                    for i in range(0, 468):
                        x = int(face_landmarks.landmark[i].x * width)
                        y = int(face_landmarks.landmark[i].y * height)
                        p = [x, y]
                        landmark_points.append([x, y])

                    # create small area in forehead
                    forehead_1,forehead_2,forehead_3,forehead_4,forehead_5,forehead_6,forehead_7,forehead_8,forehead_9,forehead_10,forehead_11,forehead_12,forehead_13,forehead_14,forehead_15,forehead_16,forehead_17,forehead_18,forehead_19,forehead_20,forehead_21,forehead_22,forehead_23,forehead_24 = forehead_define()

                    # Group each small forehead array together
                    forehead = [forehead_1, forehead_2, forehead_3, forehead_4, forehead_5, forehead_6, forehead_7, forehead_8,forehead_9,
                               forehead_10, forehead_11, forehead_12, forehead_13, forehead_14, forehead_15, forehead_16, forehead_17, forehead_18,
                               forehead_19, forehead_20, forehead_21, forehead_22, forehead_23, forehead_24]

                    # create small triangle line in each area
                    for i in range(len(forehead)):
                        cv2.polylines(image, [forehead[i]], True, (0, 255, 255), 1)

                    #create mask for each area
                    mask_forehead = []
                    for i in range(24):
                        mask = mask_define(height, width)
                        mask_forehead.append(mask)

                    # Fill roi of each area in each mask
                    for i in range(len(mask_forehead)):
                        cv2.fillPoly(mask_forehead[i], [forehead[i]], (255))

                    # collect data from each of forehead area (24 values)
                    for i in range(len(mask_forehead)):
                        crop_img = cv2.bitwise_and(image, image, mask=mask_forehead[i])
                        indices_list = np.where(np.any(crop_img != [0, 0, 0], axis=-1))
                        roi_pixel_img = crop_img[indices_list]
                        var = (roi_pixel_img == [0, 255, 255]).all(axis=-1)
                        roi_pixel_img = roi_pixel_img[~var]
                        blue_var.append(roi_pixel_img[:, 0].mean())
                        green_var.append(roi_pixel_img[:, 1].mean())
                        red_var.append(roi_pixel_img[:, 2].mean())
                    red_f.append(red_var.copy())
                    green_f.append(green_var.copy())
                    blue_f.append(blue_var.copy())
                    red_var.clear()
                    green_var.clear()
                    blue_var.clear()

                    # create array with dimention (area x num_frame)
                    red_arr = np.array(red_f.copy()).T
                    green_arr = np.array(green_f.copy()).T
                    blue_arr = np.array(blue_f.copy()).T

                    area_num,frame_num = red_arr.shape

                     # check if collected data >= 20s and each updated data should be 1s (30 frames)
                    if frame_num >= frame_time and frame_num%30 == 0:
                        red, green, blue = red_arr,green_arr,blue_arr

                        # Extract color data in each area from the whole data (need only 600 samples)
                        red_color_list,green_color_list,blue_color_list = extract_data(red,green,blue,
                                                                                       idx,frame_time,area_num)
                        # reshape it from (24,1,600) to (24,600)
                        red_color_arr_np,green_color_arr_np,blue_color_arr_np = reshape_data(red_color_list,green_color_list,blue_color_list,
                                                                                         area_num,frame_time)
                        #Find hair and skin area, return skin area
                        red_no_hair_arr,green_no_hair_arr,blue_no_hair_arr = find_hair(red_color_arr_np,green_color_arr_np,blue_color_arr_np)

                        #Find rPPG of each skin area using Modified POS
                        new_area_num,_ = red_no_hair_arr.shape
                        S_pos_avg_normal = find_rppg(red_no_hair_arr,green_no_hair_arr,blue_no_hair_arr,0.5,4,30,new_area_num,frame_time)
                        S_pos_before.append(S_pos_avg_normal)

                        # Majority vote (for narrowing the frequency band of the bandpass filter)
                        #Find hr (6s hr and move 15 frames (0.5s))
                        frame_now = int(30*subinterval_time)
                        idx_tmp = 0
                        hr_tmp = []
                        while idx_tmp <= frame_time:
                            if idx_tmp + frame_now <= frame_time:
                                hr_check_tmp = find_hr(S_pos_avg_normal[idx_tmp:idx_tmp+frame_now],fft_point,df)
                                hr_tmp.append(hr_check_tmp)
                            elif idx_tmp + frame_now > frame_time:
                                break
                            idx_tmp = idx_tmp + 15

                        #Find mode hr
                        mode_val,num_div = find_group(hr_tmp)

                        #Find new passband frequency
                        low_band = ((mode_val+(num_div/2))*0.7)/60
                        high_band = ((mode_val+(num_div/2))*1.3)/60

                        # bandpass old rPPG signal with the new frequency band
                        S_pos_avg_after = bandpass_filter(S_pos_avg_normal,low_band,high_band,30)
                        S_pos_tmp.append(S_pos_avg_after)

                        # Find final hr from the improve rPPG
                        hr_check = find_hr(S_pos_avg_after,fft_point,df)

                        #Need hr with error in +- 10% (optional)
                        if count_hr > 0:
                            if hr_check <= hr_lst[-1]*0.9 or hr_check >= hr_lst[-1]*1.1:
                                hr_lst.append(hr_lst[-1])
                            else:
                                hr_lst.append(hr_check)
                        else:
                            hr_lst.append(hr_check)
                        count_hr = 1

                        #Find positive peak for each 20 second intervals of improve rPPG signal
                        p = ppg_findpeaks_elgendi(S_pos_avg_after,30)

                        #Find time domain prv
                        mean_pp,rmssd,sdnn,sdsd = prv(p,30)

                        #Need meanPP with error in +- 10% (optional)
                        if count_mpp > 0:
                            if mean_pp <= meanPP[-1]*0.9 or mean_pp >= meanPP[-1]*1.1:
                                meanPP.append(meanPP[-1])
                            else:
                                meanPP.append(mean_pp)
                        else:
                            meanPP.append(mean_pp)
                        count_mpp = 1

                        RMSSD.append(rmssd)
                        sdNN.append(sdnn)
                        SDSD.append(sdsd)

                        #Find non linear domain prv
                        sd1,sd2,shan = prv_nonlinear(p,30)
                        sd1_lst.append(sd1)
                        sd2_lst.append(sd2)
                        sd1_sd2.append(sd1/sd2)
                        shanen.append(shan)

                        #Find freq domain prv
                        rri_in = interpolate(p,30)
                        tmp1,tmp2,tmp3 = frequency_domain(rri_in, 30)
                        hf.append(tmp1)
                        lf.append(tmp2)
                        lhf.append(tmp3)

                        #Prepare data for predicting the drowsiness
                        feature_data = []
                        if len(hr_lst) >= 10:
                            HR = add_data_s(hr_lst,data_idx)
                            feature_data.append(HR)
                            MPP = add_data_s(meanPP,data_idx)
                            feature_data.append(MPP)
                            rmssd_tmp = add_data_s(RMSSD,data_idx)
                            feature_data.append(rmssd_tmp)
                            SDNN = add_data_s(sdNN,data_idx)
                            feature_data.append(SDNN)
                            sdsd_tmp = add_data_s(SDSD,data_idx)
                            feature_data.append(sdsd_tmp)
                            SD2 = add_data_s(sd2_lst,data_idx)
                            feature_data.append(SD2)
                            SD1_SD2 = add_data_s(sd1_sd2,data_idx)
                            feature_data.append(SD1_SD2)
                            SHANEN = add_data_s(shanen,data_idx)
                            feature_data.append(SHANEN)
                            HF = add_data_s(hf,data_idx)
                            feature_data.append(HF)
                            LF = add_data_s(lf,data_idx)
                            feature_data.append(LF)
                            LHF = add_data_s(lhf,data_idx)
                            feature_data.append(LHF)
                            data_idx += 1
                            feature_arr = np.array(feature_data.copy())
                            feature_arr_re = feature_arr.reshape(-1,1)

                            # Predict the result
                            feature_transform = sc.transform(feature_arr_re.T)
                            y_pred = NN_model.predict(feature_transform, verbose=0)
                            result = check_predict(y_pred)
                            print(f"The state of the subject is: {result}")
                            if result == "Alert":
                              count_a+=1
                            else:
                              count_d+=1
                        # Update 1s data
                        idx = idx + 30

        if cv2.waitKey(20) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

#Show the ratio between alert results / total results  and drowsy results / total results
Alert_acc = count_a/(count_a+count_d) # for alert condition
print(f"Alert %: { Alert_acc*100:.2f}")
Drowsy_acc = count_d/(count_a+count_d) # for drowsy condition
print(f"Drowsy %: {Drowsy_acc*100:.2f}")